<h2>Capstone Project - The Battle of Neighborhoods - Andrew Lau</h2>
<p>
<h3>Background</h3>
There is a close relationship between China and Hong Kong. The medical system in Hong Kong is still relatively advanced and therefore it is appealing for middle-class people in China to stay in Hong Kong for medication.<p>
<h3>Objective</h3>
From the locations of the clinics in Hong Kong, we have to explore the nearby locations and classify as different useful categories for tourists in order for them to decide to stay in nearby hotels for the convenience of medication.<p>
The Hong Kong Government currently provides open data with geospatial information for easier manipulation. And the FourSquare API is useful to explore the nearby locations.<p>
A recommendation summary is provided at the end of the notebook<p>

In [114]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner

import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [115]:
#Open data from Hong Kong Government. Excel is provided
ClinicData = "http://www.dh.gov.hk/datagovhk/Clinic_under_Cap343.xlsx"

In [116]:
#Read Excel and drop irrelevant rows
df_clinic = pd.read_excel(ClinicData,skiprows=1)
df_clinic = df_clinic.drop(df_clinic.columns[[0]], axis=1)
df_clinic = df_clinic.drop([1,6,69,70,89], axis=0)
df_clinic = df_clinic.reset_index()
df_clinic = df_clinic.drop(['index'], axis=1)
df_clinic

,Name of Clinic,Address,Latitude,Longitude
0,Assemblies of God Medical Clinic,"No. 11-12, G/F, Suet Fung House, Fung Tak Esta...",22.344658,114.199971
1,BOCHK Health Care Division,"Room 1460, Bank of China Tower, 1 Garden Road,...",22.279322,114.161557
2,Caritas Clinic – Caine Road,"Medical Clinic - Room 032, G/F, Caritas House,...",22.278853,114.154302
3,Caritas Clinic – Ngau Tau Kok,"Room 1, G/F, 1 On Tak Road, Ngau Tau Kok, Kowloon",22.320880,114.216369
4,Caritas Jockey Club Tsuen Wan Clinic,"Room 3, G/F, Block B, 146 Chung On Street, Tsu...",22.368401,114.116045
5,Centre of Wellness (Kwun Tong),"Unit 7-8 (Area A), 20/F, Elite Centre, 22 Hung...",22.313368,114.219042
6,Chee Kwong Free Clinic,"297 Pai Tau Village, Sha Tin, New Territories",22.382587,114.186379
7,Ching Chung No. 2 Clinic,"28 Tsing Chung Koon Road, Tuen Mun, New Territ...",22.406277,113.974093
8,Christian Family Service Centre Medical Clinic,"Room A, G/F, 3 Tsui Ping Road, Kwun Tong, Kowloon",22.311866,114.228870
9,Clinic of China Resources Company,"D/F, Causeway Centre, 28 Harbour Road, Wan Cha...",22.280118,114.176287


In [117]:
print('There are total {} clinics in Hong Kong'.format(df_clinic.shape[0]))

There are total 85 clinics in Hong Kong


In [118]:
#Find the latitude and longitude of Hong Kong from Nominatim
address = 'Hong Kong'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hong Kong are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Hong Kong are 22.2793278, 114.1628131.


In [119]:
# set number of clusters
kclusters = 5

X = df_clinic['Latitude']
Y = df_clinic['Longitude']
Z = np.stack((X, Y), axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Z)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 4, 3, 4, 0, 1, 4, 2], dtype=int32)

In [120]:
# create map
clinic_map = folium.Map(location=[latitude, longitude], zoom_start=11)

colors = ['red', 'orange', 'yellow', 'green', 'blue']
df_clinic['Cluster'] = kmeans.labels_

for lat, long, clinic_name, cluster in zip(df_clinic['Latitude'], df_clinic['Longitude'], df_clinic['Name of Clinic'], df_clinic['Cluster']):
    label = folium.Popup(clinic_name, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(clinic_map)  

clinic_map

<p>For easier reading, a generated map image is provided for illustration<p>

<img src="https://raw.githubusercontent.com/andrewcklau/Coursera_Capstone/master/clinic_map.png">

In [121]:
#FourSquare API information
CLIENT_ID = 'UFJR3WSEYX2V0EZGL2DVJ3SIZN05OZ2VTFYG0Z1TSGQIIBQE' # your Foursquare ID
CLIENT_SECRET = 'P52B3Q4ESSJF0BDZW4501M0UBK33JTATA4CQE4IWDITCDNQD' # your Foursquare Secret
VERSION = '20180604'

#Limit to 100 entries
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UFJR3WSEYX2V0EZGL2DVJ3SIZN05OZ2VTFYG0Z1TSGQIIBQE
CLIENT_SECRET:P52B3Q4ESSJF0BDZW4501M0UBK33JTATA4CQE4IWDITCDNQD


In [122]:
#Provide a location to explore nearby locations
address = 'Hysan Place, Causeway Bay, Hong Kong'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print(latitude1, longitude1)

22.27981135 114.18378509374


In [123]:
#1000 meters is used as it is a walkable distance for tourists
radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=UFJR3WSEYX2V0EZGL2DVJ3SIZN05OZ2VTFYG0Z1TSGQIIBQE&client_secret=P52B3Q4ESSJF0BDZW4501M0UBK33JTATA4CQE4IWDITCDNQD&ll=22.27981135,114.18378509374&v=20180604&radius=1000&limit=100'

In [124]:
#Reading results from FourSquare
results = requests.get(url).json()
'There are {} around Clinic location 1'.format(len(results['response']['groups'][0]['items']))

'There are 100 around Clinic location 1'

In [125]:
#Understand the first item
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4ff44a45f31c307dc1999e9f-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d114951735',
    'name': 'Bookstore',
    'pluralName': 'Bookstores',
    'primary': True,
    'shortName': 'Bookstore'}],
  'id': '4ff44a45f31c307dc1999e9f',
  'location': {'address': '8-10/F, Hysan Place, 500 Hennessy Rd',
   'cc': 'HK',
   'city': '铜锣湾',
   'country': '香港',
   'distance': 48,
   'formattedAddress': ['8-10/F, Hysan Place, 500 Hennessy Rd', '香港'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 22.28024004574223,
     'lng': 114.18385392453827}],
   'lat': 22.28024004574223,
   'lng': 114.18385392453827},
  'name': 'Eslite Bookstore (誠品書店)',
  'photos': {'count': 0, 'groups': []}}}

In [126]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [127]:
#Filter the results with categories in FourSquare
from pandas.io.json import json_normalize

dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,state,id
0,Eslite Bookstore (誠品書店),Bookstore,"8-10/F, Hysan Place, 500 Hennessy Rd",HK,铜锣湾,香港,NaN,48,"[8-10/F, Hysan Place, 500 Hennessy Rd, 香港]","[{'lng': 114.18385392453827, 'label': 'display...",22.280240,114.183854,NaN,NaN,4ff44a45f31c307dc1999e9f
1,SOGO Freshmart (崇光超市),Supermarket,"B2/F, Sogo, 555 Hennessy Rd",HK,铜锣湾,香港,NaN,104,"[B2/F, Sogo, 555 Hennessy Rd, 香港]","[{'lng': 114.18453290190388, 'label': 'display...",22.280446,114.184533,NaN,NaN,4d13669f9844b60ce371c2d4
2,Apple Causeway Bay,Electronics Store,"銅鑼灣軒尼詩道500號, 希慎廣場, 灣仔區",HK,铜锣湾,香港,NaN,7,"[銅鑼灣軒尼詩道500號, 希慎廣場, 灣仔區, 香港]","[{'lng': 114.183808, 'label': 'display', 'lat'...",22.279751,114.183808,Causeway Bay,NaN,4ec460866c254debead58565
3,Kozy Okonomi-yaki Teppan-yaki,Japanese Restaurant,"9/F, Circle Plaza, 499 Hennessy Rd",HK,铜锣湾,香港,NaN,56,"[9/F, Circle Plaza, 499 Hennessy Rd, 香港]","[{'lng': 114.18327418474861, 'label': 'display...",22.279987,114.183274,NaN,NaN,4dfc4e0eae6033b2a4743cfa
4,Pure Fitness,Gym / Fitness Center,"15/F, Lee Theatre Plaza, 99 Percival St",HK,铜锣湾,香港,NaN,153,"[15/F, Lee Theatre Plaza, 99 Percival St, 香港]","[{'lng': 114.18391879897439, 'label': 'display...",22.278437,114.183919,NaN,NaN,50545f8ee4b0f42fb69d529b
5,UNIQLO (優衣庫),Clothing Store,"B2/F-G/F, Lee Theatre Plaza, 99 Percival St",HK,铜锣湾,香港,NaN,170,"[B2/F-G/F, Lee Theatre Plaza, 99 Percival St, 香港]","[{'lng': 114.1830157452477, 'label': 'display'...",22.278454,114.183016,Causeway Bay,NaN,4b1269b5f964a520b28923e3
6,Din Tai Fung (鼎泰豐),Dumpling Restaurant,68 Yee Wo St,HK,铜锣湾,香港,NaN,261,"[68 Yee Wo St, 香港]","[{'lng': 114.18618468728411, 'label': 'display...",22.279036,114.186185,Causeway Bay,NaN,4b3dd9a2f964a520129725e3
7,Café Corridor,Café,"G/F, 26A Russell St",HK,铜锣湾,香港,NaN,187,"[G/F, 26A Russell St, 香港]","[{'lng': 114.18240989114379, 'label': 'display...",22.278714,114.182410,NaN,NaN,4b14dd56f964a5201ea723e3
8,Pure Yoga,Yoga Studio,"25/F, Soundwill Plaza, 38 Russell St",HK,铜锣湾,香港,NaN,174,"[25/F, Soundwill Plaza, 38 Russell St, 香港]","[{'lng': 114.18260951286487, 'label': 'display...",22.278688,114.182610,NaN,NaN,4b2f15e3f964a52075e924e3
9,Urban Bakery,Bakery,"Jasons Food & Living, B2/F, Hysan Place, 500 H...",HK,铜锣湾,香港,NaN,9,"[Jasons Food & Living, B2/F, Hysan Place, 500 ...","[{'lng': 114.18387395331442, 'label': 'display...",22.279829,114.183874,NaN,NaN,5024e949e4b00bcf2c5c71c7


In [128]:
#Mapping the FourSquare categories into preferred grouping for tourist
df_group = dataframe_filtered.copy()
conditions = [
    (df_group['categories'].str.contains("Restaurant|Bar|Dessert|Noodle|Coffee|Bakery|Buffet|Pizza|Cha|Snack")),
    (df_group['categories'].str.contains("Shop|Mall|Store|store")),
    (df_group['categories'].str.contains("Supermarket")),
    (df_group['categories'].str.contains("Hotel|Hostel")),
    (df_group['categories'].str.contains("Club|Yoga|Gym"))]
choices = ['Restaurants', 'Shops', 'Supermarket', 'Hotels', 'Entertainment']
df_group['grouping'] = np.select(conditions, choices, default='Others')
df_group.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,state,id,grouping
0,Eslite Bookstore (誠品書店),Bookstore,"8-10/F, Hysan Place, 500 Hennessy Rd",HK,铜锣湾,香港,NaN,48,"[8-10/F, Hysan Place, 500 Hennessy Rd, 香港]","[{'lng': 114.18385392453827, 'label': 'display...",22.280240,114.183854,NaN,NaN,4ff44a45f31c307dc1999e9f,Shops
1,SOGO Freshmart (崇光超市),Supermarket,"B2/F, Sogo, 555 Hennessy Rd",HK,铜锣湾,香港,NaN,104,"[B2/F, Sogo, 555 Hennessy Rd, 香港]","[{'lng': 114.18453290190388, 'label': 'display...",22.280446,114.184533,NaN,NaN,4d13669f9844b60ce371c2d4,Supermarket
2,Apple Causeway Bay,Electronics Store,"銅鑼灣軒尼詩道500號, 希慎廣場, 灣仔區",HK,铜锣湾,香港,NaN,7,"[銅鑼灣軒尼詩道500號, 希慎廣場, 灣仔區, 香港]","[{'lng': 114.183808, 'label': 'display', 'lat'...",22.279751,114.183808,Causeway Bay,NaN,4ec460866c254debead58565,Shops
3,Kozy Okonomi-yaki Teppan-yaki,Japanese Restaurant,"9/F, Circle Plaza, 499 Hennessy Rd",HK,铜锣湾,香港,NaN,56,"[9/F, Circle Plaza, 499 Hennessy Rd, 香港]","[{'lng': 114.18327418474861, 'label': 'display...",22.279987,114.183274,NaN,NaN,4dfc4e0eae6033b2a4743cfa,Restaurants
4,Pure Fitness,Gym / Fitness Center,"15/F, Lee Theatre Plaza, 99 Percival St",HK,铜锣湾,香港,NaN,153,"[15/F, Lee Theatre Plaza, 99 Percival St, 香港]","[{'lng': 114.18391879897439, 'label': 'display...",22.278437,114.183919,NaN,NaN,50545f8ee4b0f42fb69d529b,Entertainment
5,UNIQLO (優衣庫),Clothing Store,"B2/F-G/F, Lee Theatre Plaza, 99 Percival St",HK,铜锣湾,香港,NaN,170,"[B2/F-G/F, Lee Theatre Plaza, 99 Percival St, 香港]","[{'lng': 114.1830157452477, 'label': 'display'...",22.278454,114.183016,Causeway Bay,NaN,4b1269b5f964a520b28923e3,Shops
6,Din Tai Fung (鼎泰豐),Dumpling Restaurant,68 Yee Wo St,HK,铜锣湾,香港,NaN,261,"[68 Yee Wo St, 香港]","[{'lng': 114.18618468728411, 'label': 'display...",22.279036,114.186185,Causeway Bay,NaN,4b3dd9a2f964a520129725e3,Restaurants
7,Café Corridor,Café,"G/F, 26A Russell St",HK,铜锣湾,香港,NaN,187,"[G/F, 26A Russell St, 香港]","[{'lng': 114.18240989114379, 'label': 'display...",22.278714,114.182410,NaN,NaN,4b14dd56f964a5201ea723e3,Others
8,Pure Yoga,Yoga Studio,"25/F, Soundwill Plaza, 38 Russell St",HK,铜锣湾,香港,NaN,174,"[25/F, Soundwill Plaza, 38 Russell St, 香港]","[{'lng': 114.18260951286487, 'label': 'display...",22.278688,114.182610,NaN,NaN,4b2f15e3f964a52075e924e3,Entertainment
9,Urban Bakery,Bakery,"Jasons Food & Living, B2/F, Hysan Place, 500 H...",HK,铜锣湾,香港,NaN,9,"[Jasons Food & Living, B2/F, Hysan Place, 500 ...","[{'lng': 114.18387395331442, 'label': 'display...",22.279829,114.183874,NaN,NaN,5024e949e4b00bcf2c5c71c7,Restaurants


In [129]:
#Summarize the categories for the location
df_grouping = df_group.groupby( ["grouping"] ).count()
df_grouping['categories']

grouping
Entertainment     4
Hotels            3
Others            5
Restaurants      65
Shops            21
Supermarket       2
Name: categories, dtype: int64

In [130]:
#Define a new function to iterate to explore the nearby location using FourSquare API
def exploreNearbyVenues(names, latitudes, longitudes):
    try:
        df_output = pd.DataFrame(columns=['Name', 'Entertainment', 'Hotels', 'Restaurants','Shops','Supermarket'])
        for name, lat, lng in zip(names, latitudes, longitudes):

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)

            # make the GET request
            results = requests.get(url).json()
            items = results['response']['groups'][0]['items']
            
            dataframe = json_normalize(items) # flatten JSON

            # filter columns
            filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
            dataframe_filtered = dataframe.loc[:, filtered_columns]

            # filter the category for each row
            dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

            # clean columns
            dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
            
            df_group = dataframe_filtered.copy()
            conditions = [
                (df_group['categories'].str.contains("Restaurant|Bar|Dessert|Noodle|Coffee|Bakery|Buffet|Pizza|Cha|Snack")),
                (df_group['categories'].str.contains("Shop|Mall|Store|store")),
                (df_group['categories'].str.contains("Supermarket")),
                (df_group['categories'].str.contains("Hotel|Hostel")),
                (df_group['categories'].str.contains("Club|Yoga|Gym"))]
            choices = ['Restaurants', 'Shops', 'Supermarket', 'Hotels', 'Entertainment']
            df_group['grouping'] = np.select(conditions, choices, default='Others')
            df_grouping = df_group.groupby( ["grouping"] ).count()
            
            #Ensure there is a value in Entertainment
            try:
                entertain_test = df_grouping['categories']['Entertainment']
            except:
                entertain_test = 0
            
            #Ensure there is a value in Supermarket
            try:
                supermarket_test = df_grouping['categories']['Supermarket']
            except:
                supermarket_test = 0

            #Ensure there is a value in Shops
            try:
                shop_test = df_grouping['categories']['Shops']
            except:
                shop_test = 0

            #Ensure there is a value in Hotels
            try:
                hotel_test = df_grouping['categories']['Hotels']
            except:
                hotel_test = 0
                
            #Ensure there is a value in Restaurants
            try:
                restaurant_test = df_grouping['categories']['Restaurants']
            except:
                restaurant_test = 0
                
            #Iterate to append data to the dataframe
            df_output.loc[(len(df_output))] = [name, entertain_test, hotel_test, restaurant_test, shop_test, supermarket_test]
    
    except:
        print(name)
        print(url)
        print(df_grouping['categories'])
        print(df_output)

    return(df_output)

In [131]:
#Using the clinic data to explore nearby locations
df_nearby = exploreNearbyVenues(df_clinic['Name of Clinic'], df_clinic['Latitude'], df_clinic['Longitude'])

In [132]:
df_nearby

,Name,Entertainment,Hotels,Restaurants,Shops,Supermarket
0,Assemblies of God Medical Clinic,0,1,29,5,0
1,BOCHK Health Care Division,11,7,41,9,1
2,Caritas Clinic – Caine Road,9,3,66,4,0
3,Caritas Clinic – Ngau Tau Kok,0,1,56,10,1
4,Caritas Jockey Club Tsuen Wan Clinic,0,2,64,16,0
5,Centre of Wellness (Kwun Tong),2,2,57,7,1
6,Chee Kwong Free Clinic,0,1,43,14,1
7,Ching Chung No. 2 Clinic,0,0,15,6,0
8,Christian Family Service Centre Medical Clinic,2,1,51,16,1
9,Clinic of China Resources Company,5,4,68,8,2


In [133]:
# Entertainment is less important for a patient
weight_entertain = 1

# A nearby hotel is important for clinic admission
weight_hotel = 5

# Nearby restaurants are important for daily meals
weight_restaurant = 3

# Shopping is less important for a patient
weight_shops = 1

# A nearby supermarket is necessary for daily needs
weight_supermarket = 3

In [134]:
#Calculate the overall score of each clinic with weighted values
df_nearby['Score'] = df_nearby['Entertainment'] * weight_entertain + df_nearby['Hotels'] * weight_hotel + df_nearby['Restaurants'] * weight_restaurant + df_nearby['Shops'] * weight_shops + df_nearby['Supermarket'] * weight_supermarket
df_nearby = df_nearby.sort_values(by=['Score'], ascending=False)
df_nearby

,Name,Entertainment,Hotels,Restaurants,Shops,Supermarket,Score
58,The Islamic Union Medical Clinic,4,5,73,8,2,262
67,Chung Sing Benevolent Society Clinic,4,3,72,14,2,255
31,Hong Kong Red Swastika Society Free Clinic,2,7,66,14,1,252
22,Hong Kong Breast Cancer Foundation Breast Heal...,2,8,65,14,0,251
20,HKFTU Medical & Healthcare Centre (Causeway Ba...,4,4,66,20,2,248
9,Clinic of China Resources Company,5,4,68,8,2,243
66,"University Health Service, The Hong Kong Polyt...",0,11,58,12,0,241
77,Kwok's Clansmen Association Living Water Clinic,1,8,60,15,0,236
68,Clinic of the Association of the Natives of Tu...,1,8,61,12,0,236
38,Lung Hing Tong Clinic,1,4,67,10,1,235


In [135]:
#Select the top 5 clinics for visualization
Top5 = [58,67,31,22,20]

df_clinic_top5 = pd.DataFrame()

for num in Top5:
    df_clinic_top5 = df_clinic_top5.append(df_clinic.loc[num])

In [136]:
df_clinic_top5

,Address,Cluster,Latitude,Longitude,Name of Clinic
58,"7/F, 40 Oi Kwan Road, Wan Chai, Hong Kong",2.0,22.277426,114.178773,The Islamic Union Medical Clinic
67,"Flat C, 1/F, 379 Lockhart Road, Wan Chai, Hong...",2.0,22.279655,114.179681,Chung Sing Benevolent Society Clinic
31,"Western Clinic, No. 25 Dragon Road, Causeway B...",2.0,22.282038,114.193087,Hong Kong Red Swastika Society Free Clinic
22,"21/F, Jupiter Tower, 9 Jupiter Street, North P...",2.0,22.286840,114.192188,Hong Kong Breast Cancer Foundation Breast Heal...
20,"Western Medicine Clinic, 1/F, 5-13 Lee Garden ...",2.0,22.280885,114.182659,HKFTU Medical & Healthcare Centre (Causeway Ba...


In [137]:
# create Top 5 map
clinic_map_top5 = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, long, clinic_name in zip(df_clinic_top5['Latitude'], df_clinic_top5['Longitude'], df_clinic_top5['Name of Clinic']):
    label = folium.Popup(clinic_name, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(clinic_map_top5)  

clinic_map_top5

For easier reading, a generated map image is used for illustration

<img src="https://raw.githubusercontent.com/andrewcklau/Coursera_Capstone/master/clinic_map_top5.png">

<h3>Conclusion</h3>
The top 5 clinics are located in the center of the Hong Kong Island which are good recommendations for the tourists in China to stay in Hong Kong. If we go back to the clustering in first map, all 5 clinics are located in the red cluster.<p>
This notebook is useful because tourists will not be familiar with the useful spots in Hong Kong, e.g. Hotels and restaurants. If they are keen to visit Hong Kong for medication, the recommended clinics will be suitable for their stays.<p>